## Making User Accounts 

In [4]:
import pymysql 

conn1 = pymysql.connect(host='127.0.0.1',port=3306,user='root',passwd='Lakshyavit_12', db='db')
cursor1 = conn1.cursor()  

conn2 = pymysql.connect(host='127.0.0.1',port=3306,user='root',passwd='Lakshyavit_12', db='backupdb') 
cursor2 = conn2.cursor() 


In [5]:
import bankSystem 

bankClient = bankSystem.Bank(conn1, cursor1, 'mainbank')
backupClient = bankSystem.Bank(conn2, cursor2, 'backupbank')

In [6]:
#Make Account
bankClient.make_account(0, 'Lakshya', 8000) 
bankClient.make_account(1, 'Anannya', 6000) 
bankClient.make_account(2, 'Abhimanyu', 5500) 

Your account was successfully made! 
Your account was successfully made! 
Your account was successfully made! 


In [4]:
# Show Accounts 
print(bankClient.get_account(0))
print(bankClient.get_account(1))
print(bankClient.get_account(2))

(0, 'Lakshya', 8000, 'No')
(1, 'Anannya', 6000, 'No')
(2, 'Abhimanyu', 5500, 'No')


## Transfer Funds (Timestamping Service)

In [5]:
def transfer_money(): 
    
    sender = int(input('Please enter ID of sender: ')) 
    receiver = int(input('Please enter ID of receiver: ')) 
    amount = int(input('Please enter amount to be sent: '))
    transferClient = bankSystem.TransferMoney(conn1, cursor1, sender, receiver, amount)
    transferClient.transfer()

In [6]:
transfer_money() 

Lakshya sent money at: 2021-12-05 17:06:31
Anannya received money from Lakshya at: 2021-12-05 17:06:33
Successful transfer


### Looking at the logs table

In [7]:
bankClient.show_logs() 

+----------+------------+---------------------+---------------------+
| Sender   | Receiver   | T_Start             | T_End               |
+==========+============+=====================+=====================+
| Lakshya  | Anannya    | 2021-12-05 17:06:31 | 2021-12-05 17:06:33 |
+----------+------------+---------------------+---------------------+



In [8]:
bankClient.show_all_accounts() 

+-----------+---------+
| Name      |   Money |
+===========+=========+
| Lakshya   |    6400 |
+-----------+---------+
| Anannya   |    7600 |
+-----------+---------+
| Abhimanyu |    5500 |
+-----------+---------+



## Show Database Server 

In [9]:
bankClient.show_database()

+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | No                 |
+------+-----------+---------+--------------------+



## Disaster Simulation

In [10]:
import Disaster as ds

disasterClient = ds.Disaster(conn1, conn2, cursor1, cursor2) 

### High Intensity Natural Disaster

In [11]:
disasterClient.simulate_earthquake('high') 

THERE WAS AN EARTHQUAKE!
All data was successfully backed up!


In [12]:
# See Backup server after Disaster 
backupClient.show_database()

+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | No                 |
+------+-----------+---------+--------------------+



In [13]:
# See Hotsite server after Disaster 
bankClient.show_database() 
print('The hotsite server was completely destroyed!') 

+------+--------+---------+--------------------+
| ID   | Name   | Money   | Intrusion Status   |
+======+========+=========+====================+
+------+--------+---------+--------------------+

The hotsite server was completely destroyed!


### Recover from High Intensity Earthquake

In [14]:
disasterClient.recover()  

All data was successfully recovered!


### Low Intensity Natural Disaster

In [15]:
disasterClient.simulate_earthquake('low') 

THERE WAS AN EARTHQUAKE!
All data was successfully backed up!


In [16]:
# See Backup server after Disaster 
backupClient.show_database()

+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | No                 |
+------+-----------+---------+--------------------+



In [17]:
# See Hotsite server after Disaster 
bankClient.show_database() 
print('Since low severity, we were able to save the Hotsite server to some extent!') 

+------+---------+---------+--------------------+
|   ID | Name    |   Money | Intrusion Status   |
+======+=========+=========+====================+
|    0 | Lakshya |    6400 | No                 |
+------+---------+---------+--------------------+
|    1 | Anannya |    7600 | No                 |
+------+---------+---------+--------------------+

Since low severity, we were able to save the Hotsite server to some extent!


### Recovery from Low Intensity Earthquake

In [18]:
disasterClient.recover() 
bankClient.show_database() 

All data was successfully recovered!
+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | No                 |
+------+-----------+---------+--------------------+



## Info Breach

Information Breach is a very common occurence in online systems. For our proposed Timestamping system, it was only natural that we included a DRP for handling an Info breach. For the purpose of demonstration, the DRP implementation has been shown in a way which uses a "Detection Intrusion System" to let us know the `id` of the person who has breached the system. That way, we can block all transactions from and to this particular person. In such a situation, our Timestamping system would output the alert, and block the account from further transactions. 

In [19]:
disasterClient.simulate_breach(2)

THERE WAS AN INFORMATION BREACH by:  ('Abhimanyu',)


In [20]:
bankClient.show_database()

+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | Yes                |
+------+-----------+---------+--------------------+



In [21]:
transfer_money()

Intruder ID:  2
RECEIVER IS AN INTRUDER! TRANSACTION BLOCKED


In [22]:
bankClient.show_database()

+------+-----------+---------+--------------------+
|   ID | Name      |   Money | Intrusion Status   |
+======+===========+=========+====================+
|    0 | Lakshya   |    6400 | No                 |
+------+-----------+---------+--------------------+
|    1 | Anannya   |    7600 | No                 |
+------+-----------+---------+--------------------+
|    2 | Abhimanyu |    5500 | Yes                |
+------+-----------+---------+--------------------+

